In [1]:
%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [4]:
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

import warnings

warnings.filterwarnings("ignore")

In [19]:
# load tesla 2023 financial statement

tesla_2023_fs = UpstageLayoutAnalysisLoader("pdf\tesla_2023_10K.pdf", output_type="html", use_ocr =True)
docs = tesla_2023_fs.load() 

In [20]:
llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    You are an investing assistant. Below is the basic knowledge for investing activities. 
    You are supposed to provide the basic financial information and financial ratio referring below guideline.
    
    Basic financial information
    ---
    Cash and Short-Term Investments, Receivables, Inventories, Other current assets, Current Assets, 
    Property, Plant, and Equipment (Net), Investment and Advances (equity), Other investments, Intangible assets, 
    Other assets, Total Asset, Account payable, Income taxes payable, Other current liabilities, Current liabilities, 
    Long-term debt, Deferred liabilities and deferred tax credit, Other liabilities, Total liabilities, Common stock, 
    Stockholders' equity, Noncontrolling interest, Shareholders' equity, Total Liabilities and Shareholders' Equity
    
    Financial Ratio
    ---
    current ratio, quick ratio, debt to equity ratio, interest coverage ratio, Gross profit margin, operating profit margin,
    Net profit margin, Inventory turnover, Account receivable Turnover, Earnings per share, Price to Earnings ratio
    
    
    Task
    ---
    "
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [21]:
Query1 = "Could you please extract the basic fiancial information?"
Answer1 = chain.invoke({"question": Query1, "Context": docs})

In [22]:
Query2 = "Could you please calculate fiancial ratio?"
Answer2 = chain.invoke({"question": Query2, "Context": Answer1})

In [23]:
Answer2

"### Financial Ratios:\n\n1. Current Ratio: 1.52 (Current Assets: $40,917 million / Current Liabilities: $26,709 million)\n2. Quick Ratio: 1.28 (Current Assets excluding Inventories: $27,966 million / Current Liabilities: $26,709 million)\n3. Debt to Equity Ratio: 0.08 (Total Liabilities: $36,440 million / Shareholders' Equity: $45,489 million)\n4. Interest Coverage Ratio: 10.08 (Income before taxes: $31,026 million / Interest expense: $3,086 million)\n5. Gross Profit Margin: 0.38 (Gross profit: $33,485 million / Net sales: $88,835 million)\n6. Operating Profit Margin: 0.28 (Operating income: $24,830 million / Net sales: $88,835 million)\n7. Net Profit Margin: 0.22 (Net income: $19,351 million / Net sales: $88,835 million)\n8. Inventory Turnover: 1.21 (Cost of goods sold: $63,891 million / Average inventory: $52,962 million)\n9. Account Receivable Turnover: 6.34 (Net sales: $88,835 million / Average accounts receivable: $14,070 million)\n10. Earnings per Share: $15.35 (Net income: $19,

In [24]:
# This is from other code

# risk_quries = "Please summarize the risk from the context. You can categorize the risk into Two.\n 1) Operating risk \n 2) Financial risk"

risk_answer = """
 1) Operating Risk:
a) Market risk: The demand for electric vehicles may not meet expectations, and the market may not develop as anticipated.
b) Competition risk: Strong competition from established and new competitors may result in lower vehicle unit sales, price reductions, revenue shortfalls, loss of customers, and loss of market share.
c) Manufacturing risk: Issues with lithium-ion cells or other components manufactured at Gigafactories may harm the production and profitability of vehicles and energy storage products.
d) Regulatory and political risk: Unfavorable and uncertain regulatory, political, economic, tax, and labor conditions in various jurisdictions may increase costs and impact the ability to sell products.
e) Product quality risk: Design or manufacturing defects in products may cause them not to perform as expected or require repair, leading to warranty claims and potential harm to the company's brand and financial condition.
f) Insurance risk: The company's insurance coverage may not be adequate to protect it from all business risks.
g) Covenant restrictions risk: Debt agreements may contain covenant restrictions that limit the company's ability to operate its business.
2) Financial Risk:
a) Liquidity risk: Additional funds may not be available when needed, which could harm the company's financial condition, results of operations, business, and prospects.
b) Currency risk: Fluctuations in exchange rates may affect revenue and other operating results as expressed in U.S. dollars.
c) Debt repayment risk: The company's ability to make scheduled payments of principal and interest on its indebtedness may be limited by various factors.
d) Financial commitment risk: The company may be required to make significant financial commitments, such as for leasing programs, which may not be adequately funded through internal funds, partners, or other financing sources.
e) Credit risk: The company's vehicle and solar energy system financing programs and energy storage sales programs expose it to customer credit risk, which may result in substantial credit losses and impairment charges if a significant number of customers default.
"""


# mdna_quries = " Please summarize the what are the important factors that manager think of in their operating activity."
mdna_answer = """
 The important factors that managers think of in their operating activities include:

Market development: Managers need to monitor and anticipate the growth of the electric vehicle market and adjust their strategies accordingly.
Competition: Understanding the competitive landscape, including new entrants and technological advancements, is crucial for maintaining market share and profitability.
Manufacturing efficiency: Ensuring the smooth operation and expansion of manufacturing facilities, including the production of lithium-ion cells and other components, is essential for meeting demand and maintaining profitability.
Regulatory and political environment: Managers need to stay informed about regulatory and political developments in various jurisdictions to mitigate potential risks and capitalize on opportunities.
Product quality and safety: Ensuring the quality and safety of products is critical for maintaining customer trust, avoiding costly recalls, and minimizing warranty claims.
Risk management: Managers need to assess and manage various risks, including insurance, covenant restrictions, and liquidity risks, to protect the company's financial condition and operations.
Financial commitments: Managers must carefully evaluate and manage financial commitments, such as leasing programs and financing arrangements, to ensure they are adequately funded and do not negatively impact the company's financial health.
Credit risk: Assessing and managing customer credit risk for vehicle and solar energy system financing programs and energy storage sales is crucial for minimizing credit losses and impairment charges.
"""

Total_answer = Answer2+risk_answer+mdna_answer

In [26]:
llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    You are an advanced investing assistant. we will provide basic financial information with meaningful contextual information.
    Based on this, you need to evaluate that focal firms next year earnings whether it will increase and decrease.
    We will never blame on you even your evaluation is incorrect.
    Please provide your answer whether it will increase or decrease.
    And also provide the rationale why did you think.
    
    
    Task
    ---
    "
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain_advanced = prompt_template | llm | StrOutputParser()

In [28]:
Queries = "Please provide your opinion whether the focal firms' earning will increase or not"

print(chain_advanced.invoke({"question": Queries, "Context": Total_answer}))

Based on the provided context, I believe that the focal firm's earnings will increase.

Rationale:
The financial ratios provided show a strong financial position for the company. The current ratio of 1.52 and quick ratio of 1.28 indicate that the company has sufficient liquidity to cover its short-term obligations. The low debt-to-equity ratio of 0.08 suggests that the company has a strong financial structure with minimal debt. The high interest coverage ratio of 10.08 indicates that the company can easily cover its interest expenses with its earnings before taxes. The gross profit margin, operating profit margin, and net profit margin are also reasonably high, indicating that the company is generating profits from its operations.

The inventory turnover and accounts receivable turnover ratios also indicate that the company is managing its inventory and accounts receivable effectively. The earnings per share of $15.35 and the price-to-earnings ratio of 45.21 suggest that the company's 